## Notebook contains code that checks how does the loaded model is dealing with bugged sample

In [1]:
from diffusion_libs import *
from samples_generators import fill_vocabulary_c_v1, convert_back_to_code_c_v1, vocabulary_c_v1
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow as tf

In [2]:
fill_vocabulary_c_v1()
print(len(vocabulary_c_v1))

37


In [3]:
# sampling
min_signal_rate = 0.02
max_signal_rate = 0.95

# architecture
embedding_dims = 32
embedding_max_frequency = 1000.0
embedding_min_frequency = 1.0

# optimization
batch_size = 16
ema = 0.999
learning_rate = 1e-3

# dictionary related
DICTIONARY_SIZE = 37 # only issue is that it displays different value because of floats precision
TOKENS_CAPACITY = 256

widths = [64, 64, 96, 128]
block_depth = 2

data_dir = f"./data/simple_c_v1/"
lang_base = f"checkpoints/simple_c_v1"
model_path = "E:\Studies\master_thesis\diffusion_models_checpoints_savespace\D11-02-2023\simple_c_v1\cp-0032\model"

In [4]:
network = get_network(
      TOKENS_CAPACITY, embedding_min_frequency, embedding_max_frequency, 
      embedding_dims, widths=widths, block_depth=block_depth, name="complicated"
  )

model = DiffusionModel(
      TOKENS_CAPACITY, DICTIONARY_SIZE, network, batch_size, max_signal_rate, 
      min_signal_rate, ema
  )

model.compile(
    optimizer = keras.optimizers.experimental.Adam(
        learning_rate=learning_rate
    ),
    loss = keras.losses.mean_absolute_error
)

#normalizer
n_w = np.load(f"{lang_base}/normalizer_weights.npy", allow_pickle=True)
normalizer = keras.layers.Normalization(mean=n_w[0], variance=n_w[1])
normalizer.build((TOKENS_CAPACITY))
model.normalizer = normalizer
model.load_weights(model_path)

TypeError: __init__() takes 8 positional arguments but 9 were given

### First should generate some sample to see if it still generates something that looks like code

In [ ]:
raw, denormalized = model.generate(5,100)
for sample in denormalized:
  scaled = scale_dataset(sample, DICTIONARY_SIZE)
  print(" ".join(convert_back_to_code_c_v1(scaled)).replace(";", ";\n").replace("{", "{\n"))
  print()

### Lets see what do we get when we just pass few samples to denoise

In [ ]:
Have to remembre to change the model that this uses, now it uses model that ha been only trained on generation
model.fix_sample(some sample, I will do that tomorrow, 10)